In [ ]:
import numpy as np
import pandas as pd


In [ ]:
!unzip Titanic_dataset.zip

Archive:  Titanic_dataset.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train.csv               
  inflating: description.txt         
  inflating: test.csv                


In [ ]:
dat = pd.read_csv('train.csv')
dat

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,NaN,S
1,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,NaN,S
2,234,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,NaN,S
3,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0000,NaN,S
4,844,0,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
707,145,0,2,"Andrew, Mr. Edgardo Samuel",male,18.0,0,0,231945,11.5000,NaN,S
708,614,0,3,"Horgan, Mr. John",male,NaN,0,0,370377,7.7500,NaN,Q
709,578,1,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
710,702,1,1,"Silverthorne, Mr. Spencer Victor",male,35.0,0,0,PC 17475,26.2875,E24,S


1.Get data

In [ ]:
def load_data():
  train_df = pd.read_csv('train.csv')
  test_df = pd.read_csv('test.csv')
  return train_df, test_df

In [ ]:
train_df, test_df = load_data()

In [ ]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,NaN,S
1,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,NaN,S
2,234,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,NaN,S
3,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0000,NaN,S
4,844,0,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,NaN,C


In [ ]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
train_df['Survived'].sum()/len(train_df['Survived'])

0.38342696629213485

2.Clean data

In [ ]:
#ตรวจดูว่าแต่ละคอลัมมี nan ไหม
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            146
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          545
Embarked         2
dtype: int64

In [ ]:
modeembark = train_df['Embarked'].mode()[0]
train_df['Embarked'].fillna(modeembark)

0      S
1      S
2      S
3      S
4      C
      ..
707    S
708    Q
709    S
710    S
711    S
Name: Embarked, Length: 712, dtype: object

In [ ]:
def clean_data(df):
  for col in df.columns:
    mode = df[col].mode()[0]
    df[col]  = df[col].fillna(mode)
  return df  

In [ ]:
clean_data(train_df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,C23 C25 C27,S
1,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,C23 C25 C27,S
2,234,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,C23 C25 C27,S
3,634,0,1,"Parr, Mr. William Henry Marsh",male,24.0,0,0,112052,0.0000,C23 C25 C27,S
4,844,0,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,C23 C25 C27,C
...,...,...,...,...,...,...,...,...,...,...,...,...
707,145,0,2,"Andrew, Mr. Edgardo Samuel",male,18.0,0,0,231945,11.5000,C23 C25 C27,S
708,614,0,3,"Horgan, Mr. John",male,24.0,0,0,370377,7.7500,C23 C25 C27,Q
709,578,1,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
710,702,1,1,"Silverthorne, Mr. Spencer Victor",male,35.0,0,0,PC 17475,26.2875,E24,S


3. Extract features

In [ ]:
def split_data_label(df):
  label = df['Survived']
  data = df.drop('Survived', axis=1)
  return data, label

In [ ]:
split_data_label(train_df)

(     PassengerId  Pclass  ...        Cabin Embarked
 0             75       3  ...  C23 C25 C27        S
 1            205       3  ...  C23 C25 C27        S
 2            234       3  ...  C23 C25 C27        S
 3            634       1  ...  C23 C25 C27        S
 4            844       3  ...  C23 C25 C27        C
 ..           ...     ...  ...          ...      ...
 707          145       2  ...  C23 C25 C27        S
 708          614       3  ...  C23 C25 C27        Q
 709          578       1  ...          E44        S
 710          702       1  ...          E24        S
 711          510       3  ...  C23 C25 C27        S
 
 [712 rows x 11 columns], 0      1
 1      1
 2      1
 3      0
 4      0
       ..
 707    0
 708    0
 709    1
 710    1
 711    1
 Name: Survived, Length: 712, dtype: int64)

In [ ]:
#จะได้ข้อมูลชุดเดิมที่ไม่มี Survived ออกมา
train_data, train_label = split_data_label(clean_data(train_df))
test_data, test_label = split_data_label(clean_data(test_df))

In [ ]:
test_data, test_label = split_data_label(clean_data(test_df))

In [ ]:
train_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,75,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,C23 C25 C27,S
1,205,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,C23 C25 C27,S
2,234,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,C23 C25 C27,S
3,634,1,"Parr, Mr. William Henry Marsh",male,24.0,0,0,112052,0.0000,C23 C25 C27,S
4,844,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,C23 C25 C27,C


In [ ]:
#เรารู้อยุ่แล้วว่าพวกตัวอักษรเครื่องจะอ่านไม่เข้าใจ จะรู้เรื่องก้ต่อเมื่อเป็นตัวเลข
#เลยสร้างฟังก์ชั่นที่เอาเฉพาะตัวเลขออกมา
def extract_feat(df):
  feat = df.copy()
  feat = feat.drop('PassengerId',axis=1)
  feat = feat._get_numeric_data()
  return feat

In [ ]:
train_feat = extract_feat(train_data)

In [ ]:
train_feat

,Pclass,Age,SibSp,Parch,Fare
0,3,32.0,0,0,56.4958
1,3,18.0,0,0,8.0500
2,3,5.0,4,2,31.3875
3,1,24.0,0,0,0.0000
4,3,34.5,0,0,6.4375
...,...,...,...,...,...
707,2,18.0,0,0,11.5000
708,3,24.0,0,0,7.7500
709,1,39.0,1,0,55.9000
710,1,35.0,0,0,26.2875


In [ ]:
test_feat = extract_feat(test_data)
test_feat

,Pclass,Age,SibSp,Parch,Fare
0,1,49.00,1,1,110.8833
1,1,20.00,0,0,30.0000
2,3,20.00,1,0,24.1500
3,1,42.00,1,0,52.5542
4,3,4.00,1,1,16.7000
...,...,...,...,...,...
174,1,64.00,0,0,26.0000
175,2,24.00,0,2,14.5000
176,3,17.00,0,0,7.1250
177,1,28.00,1,0,82.1708


4. Train model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def train_model(feat, label):
  model = GradientBoostingClassifier(random_state=0)
  model.fit(feat, label)
  return model

In [ ]:
model = train_model(train_feat, train_label)
model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

5.Evaluate model

In [ ]:
def eval_acc(prediction, actual):
  acc = sum(prediction == actual)/ len(actual)
  return acc

In [ ]:
# นำ test_feat test_label มาทดสอบ
pred = model.predict(test_feat)
eval_acc(pred, test_label)

0.6703910614525139

6.Run pipeline

In [ ]:
# เอามารวามกันแล้วสร้างฟังก์ชั่น
def run_pipeline():
  #Get data
  train_df, test_df = load_data()

  #Clean & split data
  train_data, train_label = split_data_label(clean_data(train_df))
  test_data, test_label = split_data_label(clean_data(test_df))

  #Extract feat
  train_feat = extract_feat(train_data)
  test_feat = extract_feat(test_data)

  #Train model
  model = train_model(train_feat, train_label)

  #Evaluation
  pred = model.predict(test_feat)
  eval_acc(pred, test_label)

  return acc

In [ ]:
run_pipeline()